In [ ]:
import os
import shutil
import pandas as pd
from PIL import Image
import numpy as np
#import matplotlib.pyplot as plt
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import cv2

img_rows = 150
img_cols = 224
color_type = 3
batch_size=48
epochs=300
subject='Severstal'
main_path=os.path.join("E:\\kaggle_imgs",subject)
img_path=os.path.join(main_path,"images")
data_path=os.path.join(main_path,"Data")
saved_path=os.path.join(main_path,"saved_models")
paths=[main_path, img_path,saved_path,data_path]
for fp in paths:
        print(fp)
        if not os.path.exists(fp):        
                os.mkdir(fp)
file_path=os.path.join(saved_path,subject+"200608_")
file_best=os.path.join(saved_path,subject+"200608_")

train_img_pkl=os.path.join(data_path,"train_imgs.npy")
test_img_pkl=os.path.join(data_path,"test_imgs.npy")
train_info_pkl=os.path.join(data_path,"df_train_pickle.csv")

num_classes=4

In [ ]:
for fp in paths:
        print (fp)
        if 0:
for fp in
                
        

## Configure parameters

In [ ]:
from tqdm import tqdm_notebook
import os
def get_train_info():
        df=pd.read_csv(os.path.join(data_path,"train.csv"))
        files = os.listdir(os.path.join(data_path, "train_images"))
        df_all_images=pd.DataFrame({"ImageId":files})
        df_NoDefect=df_all_images[~df_all_images.ImageId.isin(df.ImageId)]
        df_NoDefect["ClassId"]=0
        df_NoDefect["EncodedPixels"]=np.NaN
        df_train=pd.concat([df,df_NoDefect]).reset_index(drop=True)    
    
        legacy_df=pd.DataFrame(columns=["ImageId_ClassId","EncodedPixels"])
        my_group=df_train.groupby("ImageId")
        for img_id,img_df in tqdm_notebook(my_group):
                for i in range(1,5):
                        avail_classes = list(img_df.ClassId)

        row = dict()
        row['ImageId_ClassId'] = img_id + '_' + str(i)

        if i in avail_classes:
                row['EncodedPixels'] = img_df.loc[img_df.ClassId == i].EncodedPixels.iloc[0]
        else:
                row['EncodedPixels'] = np.nan

        legacy_df = legacy_df.append(row, ignore_index=True)
        return legacy_df

In [ ]:
fp=train_info_pkl
if os.path.exists(fp):
    df_train=pd.read_csv(fp)
    print("df_train load complete")
else:
    df_train=get_train_info()
    df_train['Image'] = df_train['ImageId_ClassId'].map(lambda x: x.split('_')[0])
    df_train['HavingDefection'] = df_train['EncodedPixels'].map(lambda x: 0 if x is np.nan else 1)
    df_train.to_csv(fp,index=False)

In [ ]:
image_col = np.array(df_train['Image'])
image_files = image_col[::4]
all_labels = np.array(df_train['HavingDefection']).reshape(-1, 4)

## First, check the number of each class

In [ ]:
from collections import defaultdict
import seaborn as sns
import os

In [ ]:
class_dict

In [ ]:
class_dict = defaultdict(int)

kind_class_dict = defaultdict(int)

no_defects_num = 0
defects_num = 0

for col in range(0, len(df_train), 4):
    img_names = [str(i).split("_")[0] for i in df_train.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError
        
    labels = df_train.iloc[col:col+4, 1]
    if labels.isna().all():
        no_defects_num += 1
    else:
        defects_num += 1
    
    kind_class_dict[sum(labels.isna().values == False)] += 1
        
    for idx, label in enumerate(labels.isna().values.tolist()):
        if label == False:
            class_dict[idx+1] += 1

In [ ]:
print("the number of images with no defects: {}".format(no_defects_num))
print("the number of images with defects: {}".format(defects_num))

In [ ]:
fig, ax = plt.subplots()
sns.barplot(x=list(class_dict.keys()), y=list(class_dict.values()), ax=ax)
ax.set_title("the number of images for each class")
ax.set_xlabel("class")
class_dict

In [ ]:
fig, ax = plt.subplots()
sns.barplot(x=list(kind_class_dict.keys()), y=list(kind_class_dict.values()), ax=ax)
ax.set_title("Number of classes included in each image");
ax.set_xlabel("number of classes in the image")
kind_class_dict

## check image data

In [ ]:
from pathlib import Path
train_size_dict = defaultdict(int)
train_path = Path(data_path, "train_images/")

for img_name in train_path.iterdir():
    img = Image.open(img_name)
    train_size_dict[img.size] += 1

In [ ]:
train_size_dict

In [ ]:
test_size_dict = defaultdict(int)
test_path = Path(data_path,"test_images/")

for img_name in test_path.iterdir():
    img = Image.open(img_name)
    test_size_dict[img.size] += 1

In [ ]:
test_size_dict

## Let's visualization mask!

In [ ]:
df_train

In [ ]:
palet = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]

In [ ]:
def name_and_mask(start_idx):
    col = start_idx
    img_names = [str(i).split("_")[0] for i in df_train.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError

    labels = df_train.iloc[col:col+4, 1]
    mask = np.zeros((256, 1600, 4), dtype=np.uint8)

    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            mask_label = np.zeros(1600*256, dtype=np.uint8)
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            for pos, le in zip(positions, length):
                mask_label[pos-1:pos+le-1] = 1
            mask[:, :, idx] = mask_label.reshape(256, 1600, order='F')
    return img_names[0], mask

In [ ]:
def show_mask_image(col):
    name, mask = name_and_mask(col)
    img = cv2.imread(str(train_path / name))
    fig, ax = plt.subplots(figsize=(15, 15))

    for ch in range(4):
        #contours, _ = cv2.findContours(mask[:, :, ch], cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        _,contours,_ = cv2.findContours(mask[:, :, ch].astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        for i in range(0, len(contours)):
            cv2.polylines(img, contours[i], True, palet[ch], 2)
    ax.set_title(name)
    ax.imshow(img)
    plt.show()

fig, ax = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    ax[i].axis('off')
    ax[i].imshow(np.ones((50, 50, 3), dtype=np.uint8) * palet[i])
    ax[i].set_title("class color: {}".format(i+1))
fig.suptitle("each class colors")

plt.show()

## Images with no defect

In [ ]:
df_train

In [ ]:
idx_no_defect = []
idx_class_1 = []
idx_class_2 = []
idx_class_3 = []
idx_class_4 = []
idx_class_multi = []
idx_class_triple = []

for col in range(0, len(df_train), 4):
    img_names = [str(i).split("_")[0] for i in df_train.iloc[col:col+4, 0].values]
    if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
        raise ValueError
        
    labels = df_train.iloc[col:col+4, 1]
    if labels.isna().all():
        idx_no_defect.append(col)
    elif (labels.isna() == [False, True, True, True]).all():
        idx_class_1.append(col)
    elif (labels.isna() == [True, False, True, True]).all():
        idx_class_2.append(col)
    elif (labels.isna() == [True, True, False, True]).all():
        idx_class_3.append(col)
    elif (labels.isna() == [True, True, True, False]a).all():
        idx_class_4.append(col)
    elif labels.isna().sum() == 1:
        idx_class_triple.append(col)
    else:
        idx_class_multi.append(col)

In [ ]:
for idx in idx_no_defect[:5]:
    show_mask_image(idx)

In [ ]:
for idx in idx_class_1[:5]:
    show_mask_image(idx)

In [ ]:
for idx in idx_class_2[:5]:
    show_mask_image(idx)

In [ ]:
for idx in idx_class_3[:5]:
    show_mask_image(idx)

In [ ]:
for idx in idx_class_4[:5]:
    show_mask_image(idx)

In [ ]:
for idx in idx_class_multi[:5]:
    show_mask_image(idx)

In [ ]:
for idx in idx_class_triple:
    show_mask_image(idx)

## Is there the pixel that have multi label?

In [ ]:
from tqdm import tqdm_notebook
for col in tqdm_notebook(range(0, len(train_df), 4)):
    name, mask = name_and_mask(col)
    if (mask.sum(axis=2) >= 2).any():
        show_mask_image(col)

## Draw some chars for input dataset

In [ ]:
def plot_figures(
    sizes,
    pie_title,
    start_angle,
    bar_title,
    bar_ylabel,
    labels=('Class 1', 'Class 2', 'Class 3', 'Class 4'),
    colors=None,
    explode=(0, 0, 0, 0.1),
):
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    y_pos = np.arange(len(labels))
    barlist = axes[0].bar(y_pos, sizes, align='center')
    axes[0].set_xticks(y_pos, labels)
    axes[0].set_ylabel(bar_ylabel)
    axes[0].set_title(bar_title)
    if colors is not None:
        for idx, item in enumerate(barlist):
            item.set_color(colors[idx])

    def autolabel(rects):
        """
        Attach a text label above each bar displaying its height
        """
        for rect in rects:
            height = rect.get_height()
            axes[0].text(
                rect.get_x() + rect.get_width()/2., height,
                '%d' % int(height),
                ha='center', va='bottom', fontweight='bold'
            )

    autolabel(barlist)
    
    pielist = axes[1].pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=start_angle, counterclock=False)
    axes[1].axis('equal')
    axes[1].set_title(pie_title)
    if colors is not None:
        for idx, item in enumerate(pielist[0]):
            item.set_color(colors[idx])

    plt.show()

In [ ]:
print('[THE WHOLE DATASET]')

sum_each_class = np.sum(all_labels, axis=0)
plot_figures(
    sum_each_class,
    pie_title='The percentage of each class',
    start_angle=90,
    bar_title='The number of images for each class',
    bar_ylabel='Images',
    colors=COLORS,
    explode=(0, 0, 0, 0.1)
)

sum_each_sample = np.sum(all_labels, axis=1)
unique, counts = np.unique(sum_each_sample, return_counts=True)

plot_figures(
    counts,
    pie_title='The percentage of the number of classes appears in an image',
    start_angle=120,
    bar_title='The number of classes appears in an image',
    bar_ylabel='Images',
    labels=[' '.join((str(label), 'class(es)')) for label in unique],
    explode=np.zeros(len(unique))
)

## Split dataset into training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(image_files, all_labels, test_size=TEST_SIZE, random_state=RANDOM_STATE)

In [ ]:
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_val:', X_val.shape)
print('y_val:', y_val.shape)

In [ ]:
print('[TRAINING SET]')

sum_each_class = np.sum(y_train, axis=0)
plot_figures(
    sum_each_class,
    pie_title='The percentage of each class',
    start_angle=90,
    bar_title='The number of images for each class',
    bar_ylabel='Images',
    colors=COLORS,
    explode=(0, 0, 0, 0.1)
)


sum_each_sample = np.sum(y_train, axis=1)
unique, counts = np.unique(sum_each_sample, return_counts=True)

plot_figures(
    counts,
    pie_title='The percentage of the number of classes appears in an image',
    start_angle=120,
    bar_title='The number of classes appears in an image',
    bar_ylabel='Images',
    labels=[' '.join((str(label), 'class(es)')) for label in unique],
    explode=np.zeros(len(unique))
)

## Visualize some images and corresponding labels

In [ ]:
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
def show_samples(samples):
    for sample in samples:
        fig, ax = plt.subplots(figsize=(15, 10))
        img_path = os.path.join(data_path, 'train_images', sample[0])
        img = cv2.imread(img_path).copy()

        # Get annotations
        labels = df[df['ImageId_ClassId'].str.contains(sample[0])]['EncodedPixels']

        patches = []
        for idx, rle in enumerate(labels.values):
            if rle is not np.nan:
                mask = rle2mask(rle)
                contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours:
                    poly_patch = Polygon(contour.reshape(-1, 2), closed=True, linewidth=1, edgecolor=COLORS[idx], fill=False)
                    patches.append(poly_patch)
        p = PatchCollection(patches, match_original=True, cmap=matplotlib.cm.jet)

        ax.imshow(img/255)
        ax.set_title('{} - ({})'.format(sample[0], ', '.join(sample[1].astype(np.str))))
        ax.add_collection(p)
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        plt.show()

In [ ]:
train_pairs = np.array(list(zip(X_train, y_train)))
train_samples = train_pairs[np.random.choice(train_pairs.shape[0], NUM_TRAIN_SAMPLES, replace=False), :]

a=show_samples(train_samples)

In [ ]:
img

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [ ]:
import visdom
vis=visdom.Visdom()
vis.close(env="main")

## Kaggle Immitation

In [ ]:
train_df = pd.read_csv(os.path.join(data_path,"train.csv"))
sample_df = pd.read_csv(os.path.join(data_path,"sample_submission.csv"))

In [ ]:
train_df.head()

## define value tracker

In [ ]:
def value_tracker(value_plot, value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=value,
             win = value_plot,
             update='append'
             )

## Load train data

In [ ]:
def read_train_info():
    fp=train_info_pkl
    if False and os.path.exists(fp):
        try:
            print('loading train data from csv', flush=True)
            df_train=pd.read_csv(fp)
            print('complete!', flush=True)
        except EOFError:
            print('EOFError raised.', flush=True)
        
    else:
        df_train=pd.read_csv(data_path+"/train.csv")
        df_train["CAT"]=df_train[['healthy', 'multiple_diseases', 'rust','scab']].values.argmax(axis=1)
        df_train["id"]=df_train["image_id"].apply(lambda x : int(x.split("_")[1]))        
        X=df_train['id'].values
        y=df_train['CAT'].values
        skf=StratifiedKFold(n_splits=5,random_state=22)        
        df_train["fold"]=-1
        sum=0
        for i, (trn_idx,vld_idx) in enumerate(skf.split(X,y)):
            df_train.loc[vld_idx,"fold"]=i
        df_train.to_csv(train_info_pkl,index=False)
        
    return df_train

df_train=read_train_info()


## Augmentation

In [ ]:
import torchvision.transforms as transforms

transform_train = transforms.Compose([
#    transforms.RandomCrop(32, padding=4),
    transforms.Resize((img_rows,img_cols),interpolation=Image.NEAREST),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

transform_valid = transforms.Compose([
    transforms.Resize((img_rows,img_cols),interpolation=Image.NEAREST),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.5, 1/0.5, 1/0.5 ]),
                                transforms.Normalize(mean = [ -0.5, -0.5, -0.5 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, Rotate, Cutout, HorizontalFlip, Normalize
)
from albumentations.pytorch import ToTensor, ToTensorV2
transform_train= Compose([
    Rotate(20),
    ToTensor()
])
transform_valid=Compose([
    ToTensor()
])

## Dataset

In [ ]:
from torch.utils.data import Dataset
class PlantDataset(Dataset):
    def __init__(self, df, tr=None):
        self.df = df
        self.tr=tr
        
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, idx):
        name=self.df.iloc[idx].image_id
        image_src=img_path+"/"+name+".jpg"
        image = Image.open(image_src)
        labels = self.df.iloc[idx].CAT

        if self.tr is not None:
            image=self.tr(image)
        
        return image, labels

## transforms and normalization

In [ ]:
sel=1
trn_fold=[i for i in range(5) if i not in [sel]]
val_fold=[i for i in range(5) if i in [sel]]
trn_idx=df_train[df_train.fold.isin(trn_fold)].index
val_idx=df_train[df_train.fold.isin(val_fold)].index
print(trn_idx.shape,val_idx.shape)

In [ ]:
trainset = PlantDataset(df=df_train.loc[trn_idx],
                       tr=transform_train)
validset =PlantDataset(df=df_train.loc[val_idx],
                      tr=transform_valid)

train_loader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(validset,
                                          batch_size=batch_size,
                                          shuffle=False, num_workers=0)

In [ ]:
for img,lab in train_loader:
    break

In [ ]:
plt.imshow(img[0].permute(1,2,0))

## Model

In [ ]:
# import torchvision.models.resnet as resnet

In [ ]:
# conv1x1=resnet.conv1x1
# Bottleneck = resnet.Bottleneck
# BasicBlock= resnet.BasicBlock

In [ ]:
# class ResNet(nn.Module):

#     def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
#         super(ResNet, self).__init__()
#         self.inplanes = 16
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1,
#                                bias=False)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.relu = nn.ReLU(inplace=True)
#         #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
#         self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
#         self.layer2 = self._make_layer(block, 32, layers[1], stride=1)
#         self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
#         self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Linear(128 * block.expansion, num_classes)

#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

#         # Zero-initialize the last BN in each residual branch,
#         # so that the residual branch starts with zeros, and each residual block behaves like an identity.
#         # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
#         if zero_init_residual:
#             for m in self.modules():
#                 if isinstance(m, Bottleneck):
#                     nn.init.constant_(m.bn3.weight, 0)
#                 elif isinstance(m, BasicBlock):
#                     nn.init.constant_(m.bn2.weight, 0)

#     def _make_layer(self, block, planes, blocks, stride=1):
#         downsample = None
#         if stride != 1 or self.inplanes != planes * block.expansion:
#             downsample = nn.Sequential(
#                 conv1x1(self.inplanes, planes * block.expansion, stride),
#                 nn.BatchNorm2d(planes * block.expansion),
#             )

#         layers = []
#         layers.append(block(self.inplanes, planes, stride, downsample))
#         self.inplanes = planes * block.expansion
#         for _ in range(1, blocks):
#             layers.append(block(self.inplanes, planes))

#         return nn.Sequential(*layers)

#     def forward(self, x):
#         #[-1,3,32,32]
#         x = self.conv1(x)
#         #x.shape =[1, 16, 32,32]
#         x = self.bn1(x)
#         x = self.relu(x)
#         #x = self.maxpool(x)

#         x = self.layer1(x)
#         #x.shape =[1, 128, 32,32]
#         x = self.layer2(x)
#         #x.shape =[1, 256, 32,32]
#         x = self.layer3(x)
#         #x.shape =[1, 512, 16,16]
#         x = self.layer4(x)
#         #x.shape =[1, 1024, 8,8]
        
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)

#         return x

In [ ]:
import torchvision.models  as models
import torch.nn as nn

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
resnet34 = models.resnet34(pretrained=True)
num_ftrs=resnet34.fc.in_features
resnet34.fc=nn.Linear(num_ftrs,num_classes)

In [ ]:
resnet34=resnet34.to(device)

In [ ]:
a=torch.Tensor(1,3,img_rows,img_cols).to(device)
out = resnet34(a)
print(out)

In [ ]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.SGD(resnet34.parameters(),lr=1e-2,momentum=0.9,weight_decay=5e-4)
lr_sched=optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

## make plot

In [ ]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),
                    opts=dict(title='loss_tracker', 
                              legend=['loss'], 
                              showlegend=True))
acc_plt = vis.line(Y=torch.Tensor(1).zero_(),
                   opts=dict(title='Accuracy', 
                             legend=['Acc'],
                             showlegend=True))

## define acc check func

In [ ]:
best_acc=0
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    global best_acc
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    if save and best_acc<acc:
        best_acc=acc
        torch.save(net.state_dict(), file_path+"_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

## train

In [ ]:

print(len(train_loader))
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times
    
    running_loss = 0.0
    lr_sched.step()
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet34(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(train_loader) ]))
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    
    #Check Accuracy
    acc = acc_check(resnet34, valid_loader, epoch, save=1)
    value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))
    

print('Finished Training')

In [ ]:
print(len(train_loader))

In [ ]:
torch.cuda.current_stream()

In [ ]:
torch.cuda.device_count()

In [ ]:
torch.cuda.empty_cache()